## Thumbnail Maker

Take photo, make resize into smaller images.

Typical Workflow:

1) Download image from the source location
- This is an IO task. Meanining asynchronous programming helps

2) Perform resize operation
- This is processing task, so we'll use parallel programming

3) Upload resized image to target location


Will only do first two here, as it's sufficient to illustrate

Start med single thread (nå) og del opp etterhvert. See thumbnail_maker.py (brukte spyder da Jupyter ikke funka)

 ### VIKTIG:
Se av vi over i make_thumbnails metoden har delt inn koden i to deler: 
En os-basert del "download_images()" og en CPU-basert del "perform_resizing()".
Siden man bruker ulike former av concurrency for å optimalisere OS- vs CPU-baserte
oppgaver, så lønner det seg IKKE å blande disse to typene handlinger. 
Dvs: Koden blir mindre effektiv dersom en skriver OS sammen med CPU basert oppgaver,
siden man ikke kan optimalisere begge samtidig :) 
  
TASK       METHOD    
 OS <-> Asynchronous programming
CPU <-> parallel processing

## Threads

### Process 
= the exectuion context of a running program <br>
= a running instance of a computer program <br>
Så basicly: En kode er bare en samling instruksjoner. En prosess referer til en execution av de instruksjonene. <br>

### Thread
= The smallest sequence of instruction that can be managed (scheduled and executed) by the operating system


### Program
Kan bestå av en eller flere threads. Multiple threads betyr at en process kan utføre flere oppgaver samtidig. 

### Eksmepel threads, Music program:
Thread 1: Play current song <br>
Thread 2: Figure out what the next song in queue is, and download it. <br>
Thread 3: Responder til brukerens trykk og navigering

In [1]:
import threading

def do_work(val):
    print("doing som work in thread")
    print("echo: {}".format(val))
    return

val = "text"  #1
t = threading.Thread(target = do_work, args=(val,)) #2
t.start() #3
t.join() #4 BLOCK MAIN THREAD AND TELL MAIN TO WAIT FOR "SOMETHING" TO HAPPEN -> THRD 1 (t) goes into running state
print("done") #5



doing som work in thread
echo: text
done


### Det som skjer, kronologisk:
 - MAIN THREAD STARTS
 - import ... 
 - def do_work ...
 - make variable val = "text"
 - create -------------> Thread 1 - New (The thread is in a "new" state)
 - start --------------> Thread 1 - Start (The thread is now in a "ready" state, meaning it's available for the OS to run on a CPU)                         
 - Main thread calls join. It goes from "running" to "blocked" state, making it temporarily suspended until "something happens". That "something" is here the exectuion of thread 1
 - Thread 1 goes into running state and executes the two print instructions. 
 - Thread 1 terminates when method is completed
 - This signals to the main thread that is can go into "ready" state and finish the execution.
 
 
 ### Life span of threads
 New -> Ready <--> Running --> Terminated <br>
&emsp; &emsp;&emsp;&ensp;&nbsp;   ^-- Blocked <--'

Goes from new to ready when .start() is called <br>
From there it goes back and forth between ready and running state. So when it's activly execution on a CPU it's in running, else it's in ready. <br>
If a situation arises where the thread needs to wait for a specific condition to occour before continuing execution, it goes into blocked state. For example while waiting for an IO operation to complete.
Terminates when it's done, or when an unhandled expectation occours.

## The scheduler
An operating system (OS) module. Selects the next jobs to be admitted into the system and the next process to run.
Also how long they get to run before being suspended, and what processor core they run on. <br>

The schedulerer uses an algorythm that may decide a thread should be suspended mid execution to make room for another thread waiting for processor time. It then decides which thread should be selected and performs a "context switch" to cycle the running thread out of the CPU and the selected thread in.

### Context switch
The process of saving and restoring the state of a thread or process. (Used when switching in between threads)

If selected thread is from a different process, a full process switch occours - expensive process.
However, if  it's a  thread from the safe process the thread swtich is executed.  less expensive process

### Egen counter, register og stack
#### Counter
Maintains the instructions that have been executed at a given time
#### Register
Holds data being used in computation
#### Stack
Memory set aside as scratch space for a thread. <br>
Lokale variabler i en funksjon ligger på stacken -> Ingen fare for at ulike threads ødlegger for andre threads her. 